In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import cv2
import sys
import random
sys.path.append('../')

random.seed(42)

In [2]:
from faces_clustering import Clusterer, FeatureExtractor, is_image, get_files_folder

Using TensorFlow backend.


In [29]:
df_embs = pd.read_pickle(f'../data/leg55_resnet50.pkl')
df_embs = df_embs.loc[df_embs['embeddings']!=str('-')]
df_embs.shape

/home/paulo/.local/share/virtualenvs/face_clustering-EKR70IVd/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


(9003, 3)

In [32]:
df_embs.groupby('classes')['urls'].count().describe()

count    513.000000
mean      17.549708
std        6.905470
min        2.000000
25%       12.000000
50%       18.000000
75%       23.000000
max       31.000000
Name: urls, dtype: float64

# Extracting Features

In [5]:
acomplete_urls = get_files_folder("/home/paulo/Documentos/slnp_faces_leg_55", is_image)

In [6]:
len(complete_urls)

9006

In [6]:
df_features_senet50 = FeatureExtractor('senet50').extract(complete_urls)
df_features_senet50.head(20)

,urls,embeddings
0,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.0996729, 0.028490392, 0.20510468, 0.5868249..."
1,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.086341664, 0.13112758, 0.0015521318, 0.2116..."
2,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.10442628, 0.11352078, 0.7367716, 2.401755, ..."
3,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.06998727, 3.4273074, 0.0, 2.3481488, 0.0052..."
4,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.10927686, 0.12103541, 2.4254398, 0.02788562..."
5,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.08674781, 0.022120327, 0.010512385, 0.31749..."
6,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.061542507, 0.2265746, 0.0029556362, 1.17124..."
7,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.101131305, 0.02695675, 0.2932879, 0.2321411..."
8,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.08608747, 0.031913932, 3.3949726, 1.5851948..."
9,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.03212753, 0.9086893, 0.014074973, 0.0, 3.94..."


In [7]:
df_features_senet50.to_pickle('../data/leg55_senet50.pkl')

In [8]:
df_features_resnet50 = FeatureExtractor('resnet50').extract(complete_urls)
df_features_resnet50.head(20)

,urls,embeddings
0,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[1.3244658, 0.5904787, 4.177269, 0.7942324, 1...."
1,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.0022103277, 1.0620648, 3.5930886, 0.0461376..."
2,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.31701073, 1.8052977, 1.7435782, 7.373654, 0..."
3,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.6117908, 2.5383434, 0.0, 0.03535835, 0.5126..."
4,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.08985586, 2.0157309, 6.5390806, 1.6127877, ..."
5,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.11096674, 0.7182488, 5.1844225, 4.902494, 0..."
6,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.03200272, 0.07505312, 3.836365, 0.40002552,..."
7,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[1.8421695, 0.56172615, 4.8317094, 1.4868399, ..."
8,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.13567455, 2.904518, 0.61311257, 4.9027457, ..."
9,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[0.6128481, 0.005993923, 0.0, 0.44318846, 0.90..."


In [9]:
df_features_resnet50.to_pickle('../data/leg55_resnet50.pkl')

In [10]:
df_features_vgg16 = FeatureExtractor('vgg16').extract(complete_urls)
df_features_vgg16.head(20)

,urls,embeddings
0,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[3.763019, 1.1650515, 0.5744736, 6.1935043, 9...."
1,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[1.9997711, 1.5350305, 0.13148202, 5.673743, 0..."
2,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[5.559261, 3.0951457, 0.3216881, 8.773775, 0.0..."
3,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[3.4136424, 2.7559872, 1.0755779, 7.3527594, 7..."
4,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[7.1362042, 0.9659396, 1.0193015, 6.1010485, 3..."
5,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[2.514649, 2.928906, 0.2882858, 7.5130744, 3.0..."
6,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[1.9470098, 2.9797661, 0.0, 7.3171406, 1.60305..."
7,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[4.2727046, 1.2910197, 0.3191291, 4.6945734, 7..."
8,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[5.4242935, 0.2147629, 0.0, 8.352837, 1.492800..."
9,C:/Users/paulo/Documents/FacesProject/slnp_fac...,"[17.759718, 2.1976326, 1.2993937, 4.5613794, 0..."


In [11]:
df_features_vgg16.to_pickle('../data/leg55_vgg16.pkl')

# Clustering

In [ ]:
backbone = 'vgg16'

In [ ]:
df_embeddings = pd.read_pickle(f'../data/leg55_{backbone}.pkl')

In [ ]:
df_embeddings.shape

In [ ]:
classes = df_embeddings.urls.apply(lambda x: x.split('/')[-2])

In [ ]:
valid_indexes = df_embeddings.embeddings.apply(lambda x: str(x) != '-')
df_embeddings = df_embeddings.loc[valid_indexes]

In [ ]:
df_embeddings['classes'] = classes

unique_classes = list(df_embeddings.classes.unique())

samples_classes = random.sample(unique_classes, 5)

sample_indices =  df_embeddings.classes.apply(lambda x: x in samples_classes)

In [ ]:
clusterer = Clusterer(n_clusters=514, face_embeddings = df_embeddings,algs=['kmeans', 'affinity', 'agglomerative'])

In [ ]:
clusters, models_inst = clusterer.clusterize()

In [ ]:
clusters.head()

In [ ]:
clusters.to_pickle(f'../data/leg55_clusters_{backbone}.pkl')

In [ ]:
clusters.sort_values(by=['urls']).head()

In [ ]:
models_inst['cluster_kmeans'].cluster_centers_

In [ ]:
clusters.groupby(['cluster_kmeans']).count().describe()

# Evaluating

In [ ]:
clusters = pd.read_pickle(f'../data/leg55_clusters_{backbone}.pkl')

In [ ]:
from sklearn import metrics

In [ ]:
cluster_cols = [c for c in clusters.columns if str(c).startswith('cluster')]

In [ ]:
labels_true = clusters.classes.values

In [ ]:
dict_results = {}
for col in cluster_cols:
    labels_pred = clusters[col].values
    
    dict_results[col] = [metrics.homogeneity_score(labels_true,labels_pred),
                        metrics.completeness_score(labels_true, labels_pred),
                        metrics.v_measure_score(labels_true, labels_pred)]

In [ ]:
df_results = pd.DataFrame.from_dict(dict_results, orient='index', columns=['Homogeneity Score', 'Completeness Score', 'V-Measure'])
df_results

In [ ]:
df_results.to_csv(f'../results/{backbone}_results_removed.csv')

# Checking Images that do not have embeddings

In [ ]:
df = pd.read_pickle('../data/no_valid/leg55_senet50.pkl')

In [ ]:
df.shape

In [ ]:
urls_no_embs = df.loc[(df.urls.isin(complete_urls))&(df.embeddings=='-')].urls

In [ ]:
new_urls = urls_no_embs.apply(lambda x: f"/home/paulo/Documentos/no_embs/{x.split('/')[-2]}x{x.split('/')[-1]}")

In [ ]:
extractor = FeatureExtractor('senet50')

In [ ]:
check_urls = get_files_folder("/home/paulo/Documentos/no_embs/", is_image)

In [ ]:
import os

In [ ]:
from tqdm import tqdm

for url in tqdm(urls_no_embs.values):
    
    pixels = cv2.imread(url)
    pixels_rgb = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB)
    results = extractor.detector.detect_faces(pixels_rgb)
    
    faces = []
    for result in results:
        if result['confidence'] >= 0.9:
            x1, y1, width, height = result['box']
            x2, y2 = x1 + width, y1 + height
            x1 = max(x1,0)
            y1 = max(y1,0)
            face = pixels_rgb[y1:y2, x1:x2]

            if face.shape[0] > 0 and face.shape[1] > 0:
                faces.append('cv2.resize(face, required_size)')
    if len(faces) <= 0:
        os.remove(url)
        #print(url)
        #print(results)

In [ ]:
url_series = pd.Series(complete_urls)

In [ ]:
classes = url_series.apply(lambda x: x.split('/')[-2])

In [ ]:
classes.value_counts().loc[classes.value_counts() <= 5]